In [ ]:
#pip install requests-html

In [ ]:
#pip install pandas

In [ ]:
from requests_html import *
import pandas as pd

In [15]:
from IPython.display import clear_output

###  port.hu média értékelések parszolása

In [2]:
session = HTMLSession()

port.hu review szerkezete:
```html
<div class="comment-container">
  <span class="user-rating" title="user értékelése">
   10/10
  </span>      
  <div class="message-text">
    nagyon tetszett a film!
  </div>
</div>
```
illetve:
```html
<div class="comment-container offtopic">
  <span class="user-rating" title="user értékelése">
   10/10
  </span>      
  <div class="message-text">
    esik az eső
  </div>
</div>
```

In [39]:
def parse_review(comment_container):
    comment_elem = None
    if "offtopic" not in comment_container.attrs["class"]:
        comment_elem = comment_container.find("div.message-text", first=True)
    score_elem   = comment_container.find("span.user-rating", first=True)
    if comment_elem is not None and score_elem is not None:
        return (score_elem.text.split("/")[0], comment_elem.text)
    else: return None

In [4]:
def parse_all_review(comment_container_list):
    rev_list = []
    for comment_container in comment_container_list:
        r = parse_review(comment_container)
        if r is not None:
            rev_list.append(r)
    return rev_list

utolsó oldal gombból ki lehet szedni, hány oldal van összesen:
```html
<li class="last">
    <a href="/forum/adam-almai/16107?page=13" data-page="12">&raquo;</a>
</li>
```
snippet:
```python
html.find("li.last", first=True).find("a", first=True).attrs["href"].split("=")[1]
```

In [98]:
def get_all_review(portHuLink, session, progress="100%", count=0):
    rev_list = []
    page = session.get(portHuLink)
    num_of_pages = page.html.find("li.last", first=True).find("a", first=True).attrs["href"].split("=")[1]
    for i in range(1, int(num_of_pages)+1):
        clear_output(wait=True)
        print("processing: "+portHuLink.split("/")[4]+" ("+str(i)+"/"+str(num_of_pages)+")")
        print("SUM: "+str(count+len(rev_list)))
        print("PROGRESS: "+progress)
        current_link = portHuLink+"?page="+str(i)
        current_page    = session.get(current_link)
        current_reviews = current_page.html.find("div.comment-container")
        rev_list.extend(parse_all_review(current_reviews))
    return rev_list

In [99]:
rl = get_all_review("https://port.hu/forum/adam-almai/16107", session); "TOTAL: "+str(len(rl))

processing: adam-almai (13/13)
SUM: 136
PROGRESS: 100%


'TOTAL: 137'

In [100]:
result_table = pd.DataFrame(rl, columns = ["értékelés", "szöveg"])

In [101]:
result_table.head()

,értékelés,szöveg
0,10,"Kár, hogy eddig ""patópáloskodtam""; nagyon tets..."
1,10,Időnként nem tudtam sírjak vagy nevessek....
2,10,Már kb. 1 éve felvettem és csak most néztem me...
3,10,Zseniális film!!!
4,10,"Brutál film! Nem emlékszem rá, hogy valaha is ..."


### Link lista feldolgozás

In [42]:
list_from_file = tuple(open("/home/data/links.txt", 'r'))

In [46]:
step = 0.5;progress=0

In [47]:
all_review = []

In [48]:
for link in list_from_file:
    current_link = link.replace("\n", "")
    current_reviews = get_all_review(link, session, str(progress)+"%", len(all_review))
    all_review.extend(current_reviews)
    progress+=step
"TOTAL: "+str(len(all_review))

processing: silent-hill-a-halott-varos (20/20)
SUM: 45930
PROGRESS: 99.0%


'TOTAL: 45930'

In [70]:
all_table = pd.DataFrame(all_review, columns = ["értékelés", "szöveg"])

In [71]:
all_table.head()

,értékelés,szöveg
0,7,Teljesen elment a kedvem a lélekvándorlástól. ...
1,9,"Kivancsi vagyok, sikerult-e mar a TV2 csoportn..."
2,9,"Kíváncsi vagyok, ezúttal vajon lesz-e felirat ..."
3,10,"Ez így igaz, másrészt a csupa egyéni véleménye..."
4,5,Ez egy 5 pontos film maximum. Sok szart felpon...


In [51]:
all_table.to_csv("/home/data/port_c.csv", index=False)

In [56]:
count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for index, row in all_table.iterrows():
    idx = int(row['értékelés'])-1
    count[idx]+=1
count

[2476, 600, 647, 529, 837, 753, 1210, 1956, 2434, 34488]

In [60]:
sum=0
for i in range(1, 11):
    sum+=count[i-1]*i
atl=sum/45930
atl

8.868713259307642

#### Átlag értékelés túl magas, mert jellemzően az emberek 7-10 pontra értékelnek.  Általában a 7-es értékelésben már negatív kritika (is) van, felette viszont nem nagyon, ezért 7 felett pozitívnak, a többit negatívnak veszem

In [73]:
all_table[["értékelés"]] = all_table[["értékelés"]].apply(pd.to_numeric)

In [83]:
all_table[all_table["értékelés"]==7]

,értékelés,szöveg
0,7,Teljesen elment a kedvem a lélekvándorlástól. ...
11,7,"Örülök, hogy láttam ezt a filmet. Mondanám, ho..."
72,7,"Õszinte leszek én jobbra vártam, nem igazán te..."
75,7,8 Oscar díj?:)\n\nEzzel a világ minden idõk le...
85,7,Csatlakozom. A kedvenc filmjeimet nagyon soksz...
...,...,...
45814,7,"Nem volt életem filmje, de amiért beültem rá, ..."
45840,7,"Hatalmasat csalódtam, sokkal jobbra számította..."
45849,7,"Köszönöm szépen,hogy ezt megírtad! Most felfog..."
45855,7,pyramid head a legjobb :) mar csak miatta is k...


In [ ]:
column_name = 'értékelés'
mask_pozitiv = all_table.értékelés>=8
all_table.loc[mask_jo, column_name] = "pozitív"

mask_negativ = all_table.értékelés != "pozitív"
all_table.loc[mask_negativ, column_name] = "negatív"

In [91]:
all_table.head()

,értékelés,szöveg
0,negatív,Teljesen elment a kedvem a lélekvándorlástól. ...
1,pozitív,"Kivancsi vagyok, sikerult-e mar a TV2 csoportn..."
2,pozitív,"Kíváncsi vagyok, ezúttal vajon lesz-e felirat ..."
3,pozitív,"Ez így igaz, másrészt a csupa egyéni véleménye..."
4,negatív,Ez egy 5 pontos film maximum. Sok szart felpon...


In [92]:
all_table.to_csv("/home/data/port_hu_mozes.csv", index=False)